# Restaurant Business Rankings 2020

<ol>
    <li> 
        <a href="#libraries">Importing Libraries and Loading Data</a>
    </li>
</ol>

<a id="libraries"></a>

# 1. Importing Libraries and Loading Data

In [2]:
# data handling and stats
import pandas as pd
import numpy as np
import scipy

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Uncomment cells to download the datasets

#!wget https://raw.githubusercontent.com/ftarantuviez/Restaurant-Business-Rankings-2020/master/datasets/Future50.csv -P datasets
#!wget https://raw.githubusercontent.com/ftarantuviez/Restaurant-Business-Rankings-2020/master/datasets/Independence100.csv -P datasets
#!wget https://raw.githubusercontent.com/ftarantuviez/Restaurant-Business-Rankings-2020/master/datasets/Top250.csv -P datasets

### Load datasets

In [4]:
df_top = pd.read_csv("datasets/Top250.csv")
df_ind = pd.read_csv("datasets/Independence100.csv")
df_fut = pd.read_csv("datasets/Future50.csv")

In [5]:
print("The shape of Top250 dataset is {}".format(df_top.shape))
print("The shape of Independence100 dataset is {}".format(df_ind.shape))
print("The shape of Future50 dataset is {}".format(df_fut.shape))

The shape of Top250 dataset is (250, 9)
The shape of Independence100 dataset is (100, 7)
The shape of Future50 dataset is (50, 9)


The number of observations are the expected, 250 for the 250 top restaurants, a hundred for the 100 best independece restaurants and just fifty for the future 50. However we see that the independence dataset has two column less than the other two datasets.

# 2. Top 250 restaurants

First we will start analyzing the dataset of the 250 top restaurants in the world. And to understand the data, below is a description of all the features in the dataset

- Rank: Ranking position
- Restaurant: Restaurant name
- Content: Description, only for certain restaurants
- Sales: in 2019 ($000000)
- YOY_sales: Year on year sales increase in %
- Units: Number of premises in US
- YOY_units: Year on year units increase in %
- Headquarters: Place of the restaurant's headquarters
- Segment_Category: Menu type and / or industry segment

In [6]:
df_top.head()

,Rank,Restaurant,Content,Sales,YOY_Sales,Units,YOY_Units,Headquarters,Segment_Category
0,1,McDonald's,NaN,40412,4.9%,13846,-0.5%,NaN,Quick Service & Burger
1,2,Starbucks,NaN,21380,8.6%,15049,3.0%,NaN,Quick Service & Coffee Cafe
2,3,Chick-fil-A,While Popeyes got a lot of the chicken buzz in...,11320,13.0%,2470,5.0%,NaN,Quick Service & Chicken
3,4,Taco Bell,NaN,11293,9.0%,6766,2.7%,NaN,Quick Service & Mexican
4,5,Burger King,NaN,10204,2.7%,7346,0.2%,NaN,Quick Service & Burger


## 2.1 Normalizing and Standardizing Data

To facilitate the process of data analysis and data processing is a good approach follow some standards. First we are gonna make all the columns names as lowercase

In [9]:
df_top.columns.values

"array(['Rank', 'Restaurant', 'Content', 'Sales', 'YOY_Sales', 'Units',\n       'YOY_Units', 'Headquarters', 'Segment_Category'], dtype=object)"